## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [1]:
class Matter:
    def __init__(
            self,
            purchase: float,
            storage: float,
            dry_perc: int) -> None:
        self.purchase = purchase
        self.storage = storage
        self.dry_perc = dry_perc

    @property
    def total_cost(self):
        return self.purchase+self.storage

In [2]:
import mip

m = mip.Model()

max_production = 1950000  # kWh/Quarter
energy_per_ton = 421.6      # kWh/ton
price_of_energy = 0.28      # $/kWh
max_storage = 500           # ton

farms = 5
quarters = 8

productions = [
    [700,   1500,   700,    0,  0,       700,    1500,   0],
    [1350,  0,      450,    0,  1350,    0,      450,    0],
    [0,     1500,   1500,   0,  0,       1500,   1500,   0],
    [820,   1560,   820,    0,  820,     1560,   820,    0],
    [0,     680,    1080,   0,  0,       680,    1080,   0]
]

matters = [
    Matter(0.20, 0.002, 15),
    Matter(0.18, 0.012, 28),
    Matter(0.19, 0.007, 35),
    Matter(0.21, 0.011, 37),
    Matter(0.23, 0.015, 42)
]


In [3]:
# Variables

# Amount of material into the silo per quarter (first row is full of zeros
# and represents the quarter before the first one).
# Rows are quarters, columns are farms, the sum of a row is the total 
# amount inside the silo for a quarter.
silo = [[m.add_var(var_type=mip.INTEGER) for _ in range(farms)]
        for _ in range(quarters)]
silo.insert(0, [0. for _ in range(farms)])

# Amount taken for each farm and for each quarter
taken = [[m.add_var(var_type=mip.INTEGER) for _ in range(quarters)]
         for _ in range(farms)]

# Amount used for each farm and for each quarter
used = [[m.add_var(var_type=mip.INTEGER) for _ in range(quarters)] for _ in range(
    farms)]            # rows are farms, columns are quarters



In [4]:
# Constraints

# The maximum space inside the silo is 500 tons.
for i in range(quarters):
    m.add_constr(mip.xsum(silo[i + 1]) <= max_storage)

for i in range(farms):
    for j in range(quarters):
        # The maximum amount that can be bought cannot go over the maximum farm production.
        m.add_constr(taken[i][j] <= productions[i][j])
        # Not used material is stored into the silo.
        m.add_constr(silo[j + 1][i] == silo[j][i] + taken[i][j] - used[i][j])

# The used amount cannot produce more than the maximum legal production value.
for i in range(quarters):
    m.add_constr(mip.xsum(f[i] for f in used) * energy_per_ton <= max_production)

# The mix should be between 20% and 40%.
for i in range(quarters):
    m.add_constr(mip.xsum(used[f][i] * matters[f].dry_perc for f in range(farms)) >= 20 * mip.xsum(used[f][i] for f in range(farms)))
    m.add_constr(mip.xsum(used[f][i] * matters[f].dry_perc for f in range(farms)) <= 40 * mip.xsum(used[f][i] for f in range(farms)))

In [5]:
# Objective function

# We want to maximize the revenue.
m.objective = mip.maximize(mip.xsum(used[i][j] for i in range(farms) for j in range(quarters)) * energy_per_ton * price_of_energy - mip.xsum(taken[i][j] * matters[i].purchase for i in range(farms) for j in range(quarters)) - mip.xsum(silo[i + 1][j] * matters[j].storage for i in range(quarters) for j in range(farms)))
m.optimize()
print('maximum revenue:', m.objective_value)

# revenue = [(sum(used[i][j].x for i in range(farms)) * energy_per_ton * price_of_energy - sum(taken[i][j].x * matters[i].purchase for i in range(farms))) - sum(silo[j + 1][j].x * matters[i].storage for i in range(farms)) for j in range(quarters)]
energy_per_quarter = [sum(f[i].x for f in used) * energy_per_ton for i in range(quarters)]
percentages_result = [sum(used[f][i].x * matters[f].dry_perc for f in range(farms)) / sum(used[f][i].x for f in range(farms)) for i in range(quarters) if sum(used[f][i].x for f in range(farms)) > 0]
taken_result = [[taken[j][i].x for i in range(quarters)] for j in range(farms)]
used_result = [[used[j][i].x for i in range(quarters)] for j in range(farms)]
silo_result = [[silo[i + 1][j].x for i in range(quarters)] for j in range(farms)]

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 51 (-61) rows, 84 (-36) columns and 256 (-99) elements
Clp1000I sum of infeasibilities 5.38458e-14 - average 1.0558e-15, 14 fixed columns
Coin0506I Presolve 43 (-8) rows, 62 (-22) columns and 185 (-71) elements
Clp0029I End of values pass after 43 iterations
Clp0014I Perturbing problem by 0.001% of 884.19702 - largest nonzero change 0 ( 0%) - largest zero change 2.8167904e-05
Clp0000I Optimal - objective value 2861373.9
Clp0000I Optimal - objective value 2861373.9
Coin0511I After Postsolve, objective 2861373.9, infeasibilities - dual 0 (0), primal 0 (0)
Clp0000I Optimal - objective value 2861373.9
Clp0000I Optimal - objective value 2861373.9
Clp0000I Optimal - objective value 2861373.9
Coin0511I After Postsolve, objective 2861373.9, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 2861373.92